# 第5章 决策树
1. 决策树是基于特征对实例进行分类的树形结构。
2. 决策树学习旨在构建一个与训练数据拟合好，并且复杂度小的决策树
3. 决策树算法包括三部分：特征选择、树的生成和树的剪枝。常用的算法有ID3、C4.5、CART。
4. 特征选择的目的在于选择对训练数据能正确分类的特征，常用的准则如下：
    - 样本集合D对特征A的信息增益（ID3）
    $$H(D|A) = \sum_{i=1}^n \frac{|D_i|}{D} H(D_i)$$
    $H(D)$是数据集D的熵，$H(D_i)$是数据集$D_i$的熵，$D_i$是数据集D中特征A取第 i 个值的样本子集，
    - 样本集合D对特征A的信息增益率（C4.5）
    - 样本集合D的基尼系数（CART）

In [89]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

### 例 5.1

In [2]:
# 数据集
def createdata():
    datasets = np.array([
        ['青年', '否', '否', '一般', '否'],
        ['青年', '否', '否', '好', '否'],
        ['青年', '是', '否', '好', '是'],
        ['青年', '是', '是', '一般', '是'],
        ['青年', '否', '否', '一般', '否'],
        ['中年', '否', '否', '一般', '否'],
        ['中年', '否', '否', '好', '否'],
        ['中年', '是', '是', '好', '是'],
        ['中年', '否', '是', '非常好', '是'],
        ['中年', '否', '是', '非常好', '是'],
        ['老年', '否', '是', '非常好', '是'],
        ['老年', '否', '是', '好', '是'],
        ['老年', '是', '否', '好', '是'],
        ['老年', '是', '否', '非常好', '是'],
        ['老年', '否', '否', '一般', '否'],
    ])

    feature = np.array([u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别'])

    return {'data': datasets, 'feature': feature}

## 信息增益
- 定义：特征A对训练数据集D的信心增益$g(D, A)$，定义为集合D的经验熵$H(D)$与特征A给定条件下的经验条件熵$H(D|A)$
- 理解：经验熵$H(D)$表示对数据集D分类的不确定性，经验条件熵$H(D|A)$表示在特征A给定的条件下对数据集D进行分类的不确定性。他们的差，即信息增益就表示由于特征A使得对数据集D进行分类的不确定性减少的程度。
- 结论：信息增益大的特征具有更强的分类能力。
### 算法
- ：数据集D和特征A
- ：特征A对训练数据集D的信息增益$g(D, A)$
1. 计算数据集D经验熵$H(D)$
$$H(D)=-\sum \frac{|C_k|}{|D|}\log_2 \frac{|C_k|}{|D|}$$
    - $|C_k|$属于类$C_k$的个数
    - $|D|$ 样本容量，即样本个数
2. 计算特征A对数据集D的经验条件熵$H(D|A)$
$$H(D|A) = \sum_{i=1}^n \frac{|D_i|}{D} H(D_i) = -$$

3. 计算信息增益
$$g(D, A) = H(D) - H(D|A)$$

### 延伸
数据集的经验熵相同，所以条件经验熵越小，信息增益越大

In [39]:
# 计算经验熵,
def empirical_entropy(data):
    """ 计算数据集的经验熵 """
    data_len = data.shape[0]
    labels_count = Counter(data[:, -1])    # 统计类别结果
    ent = -sum([
        (c / data_len) * np.log2(c / data_len) for c in labels_count.values()])
    return ent


# 计算条件经验熵
def cond_empirical_entropy(dataset, axis=0):
    """ 计算特征对数据集的条件经验熵 
    datasets: array, list
    """
    data_length = dataset.shape[0]
    # 某个特征取值将数据集分为与特征取值数量相同的类别
    # 统计数据集中某一列
    feature_count = Counter(dataset[:, axis])
    # 计算每个特征取值对应的数据集的经验熵
    cond_ent = []
    for feature in feature_count:
        D_i_D = feature_count[feature] / data_length
        print(D_i_D)
        indexs = dataset[:, axis] == feature
        ent = empirical_entropy(dataset[indexs])
        print(ent)
        cond_ent.append(ent * D_i_D)
    
    return sum(cond_ent)

# 信息增益
def info_gain(ent, cond_ent):
    return ent - cond_ent

# 查看数据每个特征的信息增益
# ent = empirical_entropy(xindai['data'])
# for i in range(4):
#     cond_ent = cond_empirical_entropy(xindai['data'], i)
#     
    
# 定义函数，计算信息增益
def train_info_gain(dataset):
    count = dataset['data'].shape[1] - 1
    ent = empirical_entropy(dataset['data'])   # 计算数据集的经验熵
    
    features = []
    for c in range(count):
        cond_ent = cond_empirical_entropy(dataset['data'], c)
        c_info_gain = round(info_gain(ent, cond_ent), 2)
        features.append((c, c_info_gain))
        
        print(f'当前特征：{dataset["feature"][c]},信息增益为：{c_info_gain}')
    best_ = max(features, key=lambda x: x[1])
    print(f'最大信息增益的特征为：{dataset["feature"][best_[0]]}, \
信息增益为：{best_[1]}')

In [4]:
train_info_gain(createdata())

当前特征：年龄,信息增益为：0.08
当前特征：有工作,信息增益为：0.32
当前特征：有自己的房子,信息增益为：0.42
当前特征：信贷情况,信息增益为：0.36
最大信息增益的特征为：有自己的房子, 信息增益为：0.42


## 信息增益比（率）
- 定义：特征A对训练数据集D的信息增益比$gR(D,A)$为其信息增益$g(D,A)$与训练数据集D关于特征A的值的熵$H_A(D)之比$，即：
$$gR(D,A)=\frac{g(D,A)}{H_A(D)}$$
其中$H_A(D) = - \sum_{i=1}^n \frac{|D_i|}{|D|} \log_2 \frac{|D_i|}{|D|}$

In [5]:
# 计算上例中每个特征的信息增益比
def feature_entropy(data: np.array) -> list:
    """ 计算数据集中每个特征的经验熵 
    args
        data: feature data not incloud labels
    """
    rows, cols = data.shape
    ent_list = []
    for i in range(cols):
        feature_count = Counter(data[:, i])
        ent = -sum(
            [(v / rows) * np.log2(v / rows) for v in feature_count.values()])
        ent_list.append(ent)
        
    return ent_list

def info_gain_rate(datasets) -> tuple:
    """ 计算信息增益比 """
    rows, cols = datasets.shape
    cols -= 1
    
    ent = empirical_entropy(datasets)    # 经验熵
    feature_ent = feature_entropy(datasets[:, :-1])
    # print(feature_ent)
    # 计算信息增益
    gain_list = []
    for i in range(cols):
        cond_ent = cond_empirical_entropy(datasets, i)
        gain_list.append(ent - cond_ent)
    
    # print(gain_list)
    # 计算信息增益比
    return list(np.array(gain_list) / np.array(feature_ent))


In [6]:
data = createdata()
gain_rates = info_gain_rate(data['data'])
max_idx = gain_rates.index(max(gain_rates))
print(f'数据中信息增益熵最大的特征是:{data["feature"][max_idx]}')

数据中信息增益熵最大的特征是:有自己的房子


In [7]:
for idx, value in enumerate(gain_rates):
    print(f'特征：{data["feature"][idx]},的信息增益比为：{value}')

特征：年龄,的信息增益比为：0.05237190142858302
特征：有工作,的信息增益比为：0.3524465495205019
特征：有自己的房子,的信息增益比为：0.4325380677663126
特征：信贷情况,的信息增益比为：0.23185388128724224


## ID3算法
- 核心：在决策树熵各个节点上应用信息增益准则选择特征，递归的构建决策树。
- 方法：从根节点开始，对节点计算所有可能特征的信息增益，选择信息增益最大的特征作为节点特征，由该特征的不同取值构建子节点，再对子节点递归调用上述方法。
#### 构建过程
- 输入：训练数据集D，特征集A 阈值$\epsilon$
- 输出：决策树 T。
1. 若D中所有的实例属于同一类$C_k$,则T为单节点树，并将类$C_k$作为改节点的类标记，返回T；
2. 若$A = \emptyset$，则$T$为单节点树，并将$D$中实例数最大的类$C_k$作为该节点类标记，返回$T$；
3. 否则，按之前的算法计算$A$中各特征对$D$的信息增益，选择信息增益最大的特征$A_g$；
4. 如果$A_g$的信息增益小于阈值$\epsilon$,则$T$为单节点树，将$D$中实例数最大的类$C_k$作为该节点的类标记，返回$T$；
5. 否则，对$A_g$的每一个可能值$a_i$，依$A_g=a_i$将D划分为若干非空子集$D_i$,将$D_i$中实例数最大的作为类标记，构建子节点，由节点及其子节点构成树T，返回$T_i$；
6. 对第$i$个子节点，以$D_i$为训练集，以$A-{A_g}$为特征集，递归调用步骤1-5，得到树$T_i$,返回T

In [10]:
# 定义树类
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label': self.label,
            'feature': self.feature,
            'tree': self.tree 
        }
        
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self, value, node):
        self.tree[value] = node
        
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

In [124]:
# 定义决策树
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
    
    # 计算经验熵
    @staticmethod
    def calc_ent(data: np.array) -> float:
        """ 计算熵，熵仅与信息分布有关，与其他信息无关
        data: 求一组信息的信息熵
        """
        data_len = data.shape[0]
        labels_count = Counter(data)    # 统计类别结果
        ent = -sum([
            (c / data_len) * np.log2(c / data_len) for c in labels_count.values()])
        return ent
    
    @staticmethod
    def cond_ent(x_data, labels):
        """ 计算条件经验熵 
        x_data: 训练数据集的一个特征
        y: 训练数据集对应的类标签
        """
        assert len(x_data) == len(labels)
        length = len(x_data)
        feature_count = Counter(x_data)     # 统计每个特征的数据
        
        # 计算当前特征取每个值时的数据集的经验熵H(D_i)
        feature_ent = [
            DTree.calc_ent(labels[x_data == f]) for f in feature_count.keys()]   
        # 计算该特征每个取值的频率len(D_i) / len(D)
        feature_proba = [c / length for c in feature_count.values()]

        return sum([a * b for a, b in zip(feature_ent, feature_proba)])
    
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent
    
    @staticmethod
    def info_gain_train(x_train, y_train):
        """ 计算数据集的信息增益，并找到信息增益最大的特征 
        return: best_feature, best_info_gain
        """
        assert len(x_train) == len(y_train)
        cols = x_train.shape[1]
        
        ent = DTree.calc_ent(y_train)
        cond_ents = [DTree.cond_ent(x_train[:, idx], y_train) for idx in range(cols)]
        gains = [DTree.info_gain(ent, cond) for cond in cond_ents]

        best_gain = max(gains)
        return gains.index(best_gain), best_gain
    
    def train(self, x_train, y_train):
        assert len(x_train) == len(y_train)
        features = np.array([i for i in range(x_train.shape[1])])
        
        # 1.若数据集中的所有数据都同属于一类，则T为但节点树，此类作为该节点的标记，返回树
        if len(np.unique(y_train)) == 1:
            return Node(root=True, label=y_train[0])

        # 如果特征不为空,则T为单节点树，此实例中类别数量最多作为该节点标记
        if len(features) == 0:
            # 找到数据集中类别数量最多的分类
            max_label = sorted(
                Counter(y_train).items(), reverse=True, key=lambda x: x[1])[0][0]
            return Node(root=True, label=max_label)
        
        # 3.计算最大信息增益的特征
        max_feature, max_info_gain = DTree.info_gain_train(x_train, y_train)
        max_feature_name = features[max_feature]
        
        # 4.若信息增益小于阈值，则置T为单节点树，样本实例中类别树最多的作为当前节点的标签
        if max_info_gain < self.epsilon:
            max_label = sorted(
                Counter(y_train).items(), reverse=True, key=lambda x: x[1])[0][0]
            return Node(root=True, label=max_label)
        
        # 5. 构建Ag子节点，按照特征的所有可能取值划分数据集，每个数据集的标签为其类别最多的类别
        node_tree = Node(
            root=False, feature_name=max_feature_name, feature=max_feature)
        # 获取Ag特征的所有取值
        feature_list = Counter(x_train[:, max_feature]).keys()
        # 6.移除信息增益最大的特征，并递归生成子节点
        for f in feature_list:
            drop_feature_index = x_train[:, max_feature_name] == f
            
            sub_x_train = np.delete(
                x_train[drop_feature_index], max_feature_name, axis=1)
            sub_y_train = y_train[drop_feature_index]
            
            sub_tree = self.train(sub_x_train, sub_y_train)
            node_tree.add_node(f, sub_tree)
        
        return node_tree
    
    def fit(self, x_train, y_train):
        self._tree = self.train(x_train, y_train)
        return self._tree
    
    def predict(self, x_test):
        return self._tree.predict(x_test)

In [126]:
# 测试
x_data = createdata()['data'][:, :-1]
y_data = createdata()['data'][:, -1]
td = DTree()
dt_tree = td.fit(x_data, y_data)

In [134]:
td.predict(x_data[3, :])

'是'

## C4.5算法，基于信息增益比，只需要将上例中计算信息增益部分替换为信息增益比即可

### 基尼系数

## CART算计，基于基尼系数
- 基尼指数：分类问题中，假设有K个类，样本点属于第$k$类的概率为$p_k$,则概率分布的基尼系数定义为
$$Gini(p)=\sum_{k=1}^K p_k(1-p_k) = 1 - \sum_{k=1}^K p_k^2$$
- 对于给定的样本集合D，其基尼指数为
$$Gini(D) = 1 - \sum_{k=1}^K \left(\frac{|C_k|}{|D|} \right)^2$$
- 在特征A的条件下，集合D的基尼指数定义为
$$Gini(D, A)=\frac {|D_1|}{|D|} Gini(D_1) + \frac{|D_2|}{|D|} Gini(D_2)$$
- 基尼系数$Gini(D)$,表示集合D的不确定性，基尼指数$Gini(D, A)$表述经A=a分割后集合D的不确定性，基尼指数越大，样本集合的不确定性也就越大，这一点与熵相似。

## 决策树剪枝
### 预剪枝
- 在构造决策树的同时进行剪枝。
- 在构造决策树时，一般情况下，当数据集的信息熵无法继续降低时，即停止创建决策树的分支。在预剪枝中，设定一个阈值，当决策树的信息增益小于这个阈值时，即停止剪枝。

### 后剪枝
- 决策树构建完成后进行剪枝，
- 剪枝过程中对拥有同样父节点的一组节点进行检查，判断如果将其合并熵的增加量是否小于某一阈值，如果确实小，则这一组节点可以合并为一个节点

### 后剪枝算法

In [135]:
def gini(x_train, y_train):
    """ 计算数据集的基尼系数"""
    pass

In [92]:
t

array([1, 0, 0, 1, 1])